In [1]:
import pandas as pd
import numpy as np

In [2]:
movies_data_df = pd.read_csv('data/experiment_data/movies.csv')
movies_data_df.rename(columns={'id': 'movieId'}, inplace=True)
movies_index = np.sort(movies_data_df['movieId'].values)

In [3]:
trainset_df = pd.read_csv('data/experiment_data/trainset.csv')
trainset_df = pd.merge(trainset_df, movies_data_df, how='outer', on='movieId')

In [4]:
predictedset_df = pd.read_csv('data/experiment_data/predicted_values.csv')
predictedset_df = pd.merge(predictedset_df, movies_data_df, how='outer', on='movieId')

# Inicio del ejemplo

Vamos a mostrar para un usuario y una recomendación, un ejemplo de cómo explicar la recomendacion. El ejemplo que vamos a utilizar es el usuario 1 y la película recomendada 223.,

In [5]:
## Input: insertar el usario, la película recuperada y el número de películas que queremos recuperar para cada ejemplo (k)

user = 1
movie_recommended = 223
k = 5

movie_recommended_data = predictedset_df[predictedset_df['userId'] == user][predictedset_df['movieId'] == movie_recommended]
movie_recommended_data = movie_recommended_data.drop(columns=['userId', 'language', 'country', 'duration', 'rating'])
columns_order = ['movieId', 'movie_title', 'title_year', 'director_name', 'stars', 'predicted']
movie_recommended_data = movie_recommended_data.reindex(columns=columns_order)
movie_recommended_data

/Users/jljorro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,movieId,movie_title,title_year,director_name,stars,predicted
0,223,Clerks,1994,Kevin Smith,Jason Mewes|Jeff Anderson|Jeff Anderson,4.042081


## Comparamos con los ratings más altos

En primer lugar, vamos a comparar la película recomendada con las películas que tienen un 4.0 o más según el usuario.

In [6]:
user_watched_movies = trainset_df[trainset_df['userId'] == user]
user_likes_movies = user_watched_movies[trainset_df['rating'] >= 4.0]
user_likes_movies.sort_values(by='rating', ascending=False)[:k]

/Users/jljorro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,userId,movieId,rating,timestamp,director_name,duration,genres,movie_title,language,country,title_year,stars
3262,1,596,5.0,964982838,Norman Ferguson,88,Animation|Family|Fantasy|Musical,Pinocchio,English,USA,1940,Mel Blanc|Cliff Edwards|Cliff Edwards
370,1,50,5.0,964982931,Bryan Singer,106,Crime|Drama|Mystery|Thriller,The Usual Suspects,English,USA,1995,Kevin Spacey|Kevin Pollak|Kevin Pollak
611,1,101,5.0,964980868,Wes Anderson,91,Comedy|Crime|Drama,Bottle Rocket,English,USA,1996,Andrew Wilson|Lumi Cavazos|Lumi Cavazos
840,1,151,5.0,964984041,Michael Caton-Jones,139,Adventure|Biography,Rob Roy,English,USA,1995,Liam Neeson|Eric Stoltz|Eric Stoltz
879,1,163,5.0,964983650,Robert Rodriguez,104,Action|Crime|Thriller,Desperado,English,USA,1995,Quentin Tarantino|Salma Hayek|Salma Hayek


In [7]:
movies_similarity = pd.read_csv('data/similarity_data/sim_cosine_binary.csv')

In [8]:
most_like_movies = user_likes_movies.sort_values(by='rating', ascending=False)[:k]['movieId'].values
most_like_index = np.where(np.isin(movies_index, most_like_movies))

# Index de la película recomendada
movie_recommended_index = np.where(movies_index == movie_recommended)[0][0]
movies_similarity.iloc[movie_recommended_index].values[most_like_index]

array([0.6645898 , 0.79587585, 0.46547752, 0.64644661, 0.75      ])

In [9]:
example_1_df = user_likes_movies.sort_values(by='rating', ascending=False)[:k]
example_1_df = example_1_df.drop(columns=['userId', 'timestamp', 'language', 'country', 'duration'])
columns_order = ['movieId', 'movie_title', 'title_year', 'director_name', 'stars', 'rating']
example_1_df = example_1_df.reindex(columns=columns_order)
example_1_df['similarity'] = movies_similarity.iloc[movie_recommended_index].values[most_like_index]
example_1_df.sort_values(by='similarity', ascending=False)

,movieId,movie_title,title_year,director_name,stars,rating,similarity
370,50,The Usual Suspects,1995,Bryan Singer,Kevin Spacey|Kevin Pollak|Kevin Pollak,5.0,0.795876
879,163,Desperado,1995,Robert Rodriguez,Quentin Tarantino|Salma Hayek|Salma Hayek,5.0,0.750000
3262,596,Pinocchio,1940,Norman Ferguson,Mel Blanc|Cliff Edwards|Cliff Edwards,5.0,0.664590
840,151,Rob Roy,1995,Michael Caton-Jones,Liam Neeson|Eric Stoltz|Eric Stoltz,5.0,0.646447
611,101,Bottle Rocket,1996,Wes Anderson,Andrew Wilson|Lumi Cavazos|Lumi Cavazos,5.0,0.465478


## Comparamos con más similares según las propiedades de las películas

Recuperamos las K películas más similares a la película recomendada y que ha visto el usuario.

In [10]:
# Index de la película recomendada
movie_recommended_index = np.where(movies_index == movie_recommended)[0][0]

# Index de las películas que ha visto
user_watched_index = np.where(np.isin(movies_index, user_watched_movies['movieId'].values))

# Recuperamos las k películas más similares
most_similar_movies = movies_similarity.iloc[movie_recommended_index].iloc[user_watched_index]
most_similar_movies_index = np.argsort(most_similar_movies)[-k:].values
print(most_similar_movies[most_similar_movies_index])

39     0.764298
128    0.764298
0      0.776393
124    0.786799
36     0.795876
Name: 68, dtype: float64


In [11]:
user_watched_movies.iloc[most_similar_movies_index]

,userId,movieId,rating,timestamp,director_name,duration,genres,movie_title,language,country,title_year,stars
633,1,110,4.0,964982176,Mel Gibson,178,Biography|Drama|History|War,Braveheart,English,USA,1995,Mhairi Calvey|James Robinson|James Robinson
2295,1,480,4.0,964982346,Steven Spielberg,127,Adventure|Sci-Fi|Thriller,Jurassic Park,English,USA,1993,Wayne Knight|Bob Peck|Bob Peck
0,1,1,4.0,964982703,John Lasseter,74,Adventure|Animation|Comedy|Family|Fantasy,Toy Story,English,USA,1995,Tom Hanks|Jim Varney|Jim Varney
2118,1,457,5.0,964981909,Andrew Davis,130,Action|Adventure|Crime|Drama|Mystery|Thriller,The Fugitive,English,USA,1993,Harrison Ford|Sela Ward|Sela Ward
611,1,101,5.0,964980868,Wes Anderson,91,Comedy|Crime|Drama,Bottle Rocket,English,USA,1996,Andrew Wilson|Lumi Cavazos|Lumi Cavazos


In [12]:
example_2_df = user_watched_movies.iloc[most_similar_movies_index]
example_2_df = example_2_df.drop(columns=['userId', 'timestamp', 'language', 'country'])
columns_order = ['movieId', 'movie_title', 'title_year', 'director_name', 'stars', 'rating']
example_2_df = example_2_df.reindex(columns=columns_order)
example_2_df['similarity'] = most_similar_movies[most_similar_movies_index].values
example_2_df.sort_values(by='similarity', ascending=False)

,movieId,movie_title,title_year,director_name,stars,rating,similarity
611,101,Bottle Rocket,1996,Wes Anderson,Andrew Wilson|Lumi Cavazos|Lumi Cavazos,5.0,0.795876
2118,457,The Fugitive,1993,Andrew Davis,Harrison Ford|Sela Ward|Sela Ward,5.0,0.786799
0,1,Toy Story,1995,John Lasseter,Tom Hanks|Jim Varney|Jim Varney,4.0,0.776393
633,110,Braveheart,1995,Mel Gibson,Mhairi Calvey|James Robinson|James Robinson,4.0,0.764298
2295,480,Jurassic Park,1993,Steven Spielberg,Wayne Knight|Bob Peck|Bob Peck,4.0,0.764298


## Recuperamos las más similares según Q del usuario


In [13]:
q_movies_similarity = pd.read_csv('data/similarity_data/sim_cosine_user_1.csv')

In [14]:
# Index de la película recomendada
movie_recommended_index = np.where(movies_index == movie_recommended)[0][0]

# Index de las películas que ha visto
user_watched_index = np.where(np.isin(movies_index, user_watched_movies['movieId'].values))

# Recuperamos las k películas más similares
q_most_similar_movies = q_movies_similarity.iloc[movie_recommended_index].iloc[user_watched_index]
q_most_similar_movies_index = np.argsort(q_most_similar_movies)[-k:].values
print(q_most_similar_movies[q_most_similar_movies_index])

71    0.784912
39    0.789173
73    0.834609
0     0.842373
49    0.858860
Name: 68, dtype: float64


In [15]:
example_3_df = user_watched_movies.iloc[q_most_similar_movies_index]
example_3_df = example_3_df.drop(columns=['userId', 'timestamp', 'language', 'country'])
columns_order = ['movieId', 'movie_title', 'title_year', 'director_name', 'stars', 'rating']
example_3_df = example_3_df.reindex(columns=columns_order)
example_3_df['similarity'] = q_most_similar_movies[q_most_similar_movies_index].values
example_3_df.sort_values(by='similarity', ascending=False)

,movieId,movie_title,title_year,director_name,stars,rating,similarity
840,151,Rob Roy,1995,Michael Caton-Jones,Liam Neeson|Eric Stoltz|Eric Stoltz,5.0,0.858860
0,1,Toy Story,1995,John Lasseter,Tom Hanks|Jim Varney|Jim Varney,4.0,0.842373
1061,235,Ed Wood,1994,Tim Burton,Johnny Depp|Martin Landau|Martin Landau,4.0,0.834609
633,110,Braveheart,1995,Mel Gibson,Mhairi Calvey|James Robinson|James Robinson,4.0,0.789173
941,231,Dumb & Dumber,1994,Peter Farrelly,Lauren Holly|Teri Garr|Teri Garr,5.0,0.784912
